In [16]:
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
import matplotlib.pyplot as plt

In [2]:
datapath = r"C:\Users\gpaiva\Desktop\Massa_Inteligencia.xlsx"
data = pd.read_excel(datapath)

In [3]:
class TratarCaracteresEspeciaisTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        X_transformed = X.copy()
        X_transformed = X_transformed.str.replace('[/<>()|\+\-\$%&#@\'*\"]+', ' ', regex=True)
        X_transformed = X_transformed.str.replace('[,.:;!?]+', ' ', regex=True)
        return X_transformed
    
    def fit(self, X, y=None):
        return self

stop_words = ['em','sao','ao','de','da','do','das','no',
              'ou','a','o','os','as','um','uns','uma','umas']

In [4]:
pipeline = Pipeline([
        ('tratar_caracteres', TratarCaracteresEspeciaisTransformer()),
        ('vectorize', CountVectorizer(ngram_range=(1,2), stop_words=stop_words)),
        ('score', TfidfTransformer()),
        ('svm',svm.SVC(kernel='linear',probability=True)),  
    ])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data['Exemplo'], data["Classificacao"], test_size=0.2, random_state=42)

In [6]:
model = pipeline.fit(X_train,y_train)

In [7]:
y_pred = model.predict(X_test)
prob = model.predict_proba(X_test)
conf = [max(prob[i]) for i in range(len(prob))]

In [8]:
output = {'Exemplo': X_test, 'Classificacao': y_test, 'Pred': y_pred, 'Conf': conf}
output = pd.DataFrame(output)
output['Conf'] = (output['Conf']).astype(float)
output['predict'] = output.apply(lambda row: 1 if row['Classificacao'] == row['Pred'] else 0, axis=1)
output.head()

,Exemplo,Classificacao,Pred,Conf,predict
25609,NUTELLA B-READY 1X36X4,mercearia__alimentos_secos__matinais/lanches,mercearia__alimentos_secos__matinais/lanches,0.995585,1
31738,REFRI GUARANA ANTARTICA 1L PET,bebidas__nao_alcoolicas__sucos/refrigerantes/cha,bebidas__nao_alcoolicas__sucos/refrigerantes/cha,0.999515,1
5346,BICARBONATO DE SODIO PIRATA 30G,mercearia__alimentos_secos__molhos/_temperos/_...,mercearia__alimentos_secos__molhos/_temperos/_...,0.999148,1
46074,ap barb presto ultragrip fem 2un,higiene_e_beleza__cosmetica/cuidado_pessoal__c...,higiene_e_beleza__cosmetica/cuidado_pessoal__c...,0.998417,1
50597,bolo chocolate unidade,pereciveis__frescos__padaria/_doceria/_rotisseria,pereciveis__frescos__padaria/_doceria/_rotisseria,0.998794,1


No contexto do problema de negócio, um dos principais objetivos era obter uma base de dados o mais limpa possível, minimizando as classificações falsas positivas. Para alcançar esse objetivo, foi realizada uma análise de threshold (limiar) para determinar o ponto que maximizava os valores de precisão, recall e F1-score. Nessa análise, as saídas foram convertidas em duas categorias: 0 (classificação errada) e 1 (classificação correta).

Essa análise de threshold nos forneceu uma maneira eficaz de otimizar o desempenho do modelo, garantindo que as classificações incorretas fossem minimizadas, o que é especialmente importante em problemas em que a precisão e a confiabilidade das classificações são críticas para o sucesso das operações e dos processos de negócio.

In [17]:
def best_treshold(probabilidades,y_true):
    import warnings

    # Desativar todos os avisos
    warnings.filterwarnings("ignore")
    
    # Definir uma lista de thresholds a serem avaliados
    thresholds = [i/100 for i in range(1, 100)]

    # Listas para armazenar os resultados
    results = []
    columns = ['Threshold', 'Precision', 'Recall', 'F1-score']

    # Iterar sobre os thresholds e calcular as métricas de desempenho
    for threshold in thresholds:
        # Aplicar o threshold para obter as previsões
        y_pred_encod = [1 if prob >= threshold else 0 for prob in probabilidades]

        # Calcular as métricas de desempenho
        precision = metrics.precision_score(y_true, y_pred_encod, average='weighted')
        recall = metrics.recall_score(y_true, y_pred_encod, average='weighted')
        f1_score = metrics.f1_score(y_true, y_pred_encod, average='weighted')

        # Adicionar os resultados à lista
        results.append([threshold, precision, recall, f1_score])

    # Criar o DataFrame com os resultados
    df_results = pd.DataFrame(results, columns=columns)
    return df_results

In [18]:
validacao_tresholds = best_treshold(output['Conf'],output['predict'])
validacao_tresholds.loc[validacao_tresholds['F1-score'].idxmax()]

Threshold    0.530000
Precision    0.956767
Recall       0.962253
F1-score     0.958858
Name: 52, dtype: float64